In [1]:
from backtesting.test import EURUSD, SMA
import sys
sys.path.insert(1, '/home/kenny/algotrading/model_training')
from dataset import Data
from experiments import EXPERIMENT_ID_TO_FEATURES
import pandas as pd

# data = EURUSD.copy()

model_path = '/home/kenny/algotrading/model_training/model.onnx'
features_24 = EXPERIMENT_ID_TO_FEATURES[24]

In [2]:
data = pd.read_csv('example.csv')
data['Open_Time'] = pd.to_datetime(data['Open_Time'])
data.set_index("Open_Time", inplace=True)

In [3]:
df = Data(data)
df.create_features()
df.create_target(task_type='classification')

In [4]:
df.df

,Open,High,Low,Close,Volume,RSI_14,SMA_3,EMA_3,SMA_5,EMA_5,...,low_3,close_4,open_4,high_4,low_4,close_5,open_5,high_5,low_5,Target
Open_Time,,,,,,,,,,,,,,,,,,,,,
2023-01-10 22:25:00,17438.09,17439.45,17435.26,17436.41,80.09156,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2023-01-10 22:26:00,17436.41,17437.01,17426.76,17428.62,197.24285,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2023-01-10 22:27:00,17428.62,17429.90,17419.92,17425.81,408.72043,NaN,17430.280000,17429.162500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2023-01-10 22:28:00,17425.81,17433.30,17422.76,17432.94,124.77102,NaN,17429.123333,17431.051250,NaN,NaN,...,17435.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2023-01-10 22:29:00,17432.94,17439.24,17432.47,17438.11,97.83412,NaN,17432.286667,17434.580625,17432.378,17433.865802,...,17426.76,17436.41,17438.09,17439.45,17435.26,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-13 19:23:00,19285.51,19287.90,19281.29,19282.56,115.00346,50.715762,19279.453333,19279.248534,19273.220,19276.398099,...,19257.29,19268.82,19264.38,19269.70,19257.97,19263.90,19265.30,19269.83,19255.16,1.0
2023-01-13 19:24:00,19282.56,19292.92,19275.51,19291.92,150.14374,54.356963,19286.266667,19285.584267,19277.840,19281.572066,...,19254.08,19258.92,19267.94,19271.94,19257.29,19268.82,19264.38,19269.70,19257.97,0.0
2023-01-13 19:25:00,19291.92,19302.32,19282.20,19294.47,297.14452,55.325345,19289.650000,19290.027134,19284.950,19285.871377,...,19270.74,19271.48,19258.92,19272.77,19254.08,19258.92,19267.94,19271.94,19257.29,0.0


In [21]:
import numpy as np


def get_X(data):
    """Return model design matrix X"""
    return data.filter(like='X').values


def get_y(data):
    """Return dependent variable y"""
    y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    y[y > 0] = 1
    y[y < 0] = -1
    return y


def get_clean_Xy(df):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(df)
    y = get_y(df).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

In [11]:
from backtesting import Backtest, Strategy
import onnxruntime as rt
import numpy as np
import pandas as pd

class MLTrainOnceStrategy(Strategy):
    price_delta = .004  # 0.5%

    def init(self):        
        self.session = rt.InferenceSession(model_path)
        self.input_name = self.session.get_inputs()[0].name
        self.features = features_24

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self):

        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        current_time = self.data.index[-1]

        # Forecast the next movement
        # X = get_X(self.data.df.iloc[-1:])
        X = Data(self.data.df)
        X.create_features()
        forecast = self.session.run( None, {self.input_name: X.df[self.features][-1:].values.astype(np.float32)})[0][0]

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.
        upper, lower = close[-1] * (1 + np.r_[1, -1]*self.price_delta)

        if forecast == 1 and not self.position.is_long:
            self.buy(size=.2, tp=upper, sl=lower)
        elif forecast == -1 and not self.position.is_short:
            self.sell(size=.2, tp=lower, sl=upper)

        # Additionally, set aggressive stop-loss on trades that have been open 
        # for more than two days
        for trade in self.trades:
            if current_time - trade.entry_time > pd.Timedelta('2 days'):
                if trade.is_long:
                    trade.sl = max(trade.sl, low)
                else:
                    trade.sl = min(trade.sl, high)


bt = Backtest(df.df, MLTrainOnceStrategy, commission=.001, margin=.05)
output = bt.run()

In [8]:
bt.plot()

Row(id='1475', ...)

In [12]:
output

Start                     2023-01-10 22:25:00
End                       2023-01-13 19:27:00
Duration                      2 days 21:02:00
Exposure Time [%]                   76.345643
Equity Final [$]                   6251.75728
Equity Peak [$]                   10018.35162
Return [%]                         -37.482427
Buy & Hold Return [%]               10.734492
Return (Ann.) [%]                      -100.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -38.443683
Avg. Drawdown [%]                   -9.863341
Max. Drawdown Duration        2 days 18:36:00
Avg. Drawdown Duration        0 days 17:15:00
# Trades                                  169
Win Rate [%]                        33.727811
Best Trade [%]                       0.326014
Worst Trade [%]                      -0.95959
Avg. Trade [%]                    